In [1]:
import pandas as pd
import numpy as np


In [2]:
input_path = "data/sp500_parsed.csv"
output_path = "data/sp500_parsed_filled.csv"

# 1. Load
df = pd.read_csv(input_path)

# 2. Parse dates and set as index
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date").sort_index()

# 3. Reindex to all calendar days between min and max date
all_days = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df = df.reindex(all_days)

# 4. Interpolate linearly over time for price columns
price_cols = ["open", "high", "low", "close", "adj_close"]
df[price_cols] = df[price_cols].interpolate(method="time")

# (Optional) If there are still NaNs at very start/end (no neighbors), you can keep them or forward/back fill:
# df[price_cols] = df[price_cols].ffill().bfill()

# 5. Restore date as a column
df = df.reset_index().rename(columns={"index": "date"})

# 6. Save
df.to_csv(output_path, index=False)

print(f"Saved filled data with interpolated prices to {output_path}")

Saved filled data with interpolated prices to data/sp500_parsed_filled.csv


In [4]:
solana_prices = pd.read_csv("data/trading_prices_SOLUSDT.csv")
ethereum_prices = pd.read_csv("data/trading_prices_ETHUSDT.csv")
bitcoin_prices = pd.read_csv("data/trading_prices_BTCUSDT.csv")
sp500_prices = pd.read_csv("data/sp500_parsed_filled.csv")

# Convert dates to datetime for proper comparison
solana_prices['date'] = pd.to_datetime(solana_prices['date'])
ethereum_prices['date'] = pd.to_datetime(ethereum_prices['date'])
bitcoin_prices['date'] = pd.to_datetime(bitcoin_prices['date'])
sp500_prices['date'] = pd.to_datetime(sp500_prices['date'])

# Ensure numeric columns are actually numeric
for df in [solana_prices, ethereum_prices, bitcoin_prices]:
    for col in ["8h_change", "16h_change", "24h_change", "close_price", "volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["open", "high", "low", "close", "adj_close"]:
    sp500_prices[col] = pd.to_numeric(sp500_prices[col], errors="coerce")

combined_rows = []

# Iterate over solana_prices since we want SOL data as the base
for _, row in solana_prices.iterrows():
    date = row['date']
    sol_8h_change = row['8h_change']
    sol_16h_change = row['16h_change']
    sol_24h_change = row['24h_change']
    sol_close_price = row['close_price']
    sol_volume = row['volume']

    ts_1d_ago = date - pd.Timedelta(days=1)
    ts_2d_ago = date - pd.Timedelta(days=2)
    ts_1w_ago = date - pd.Timedelta(days=7)

    sol_price_tdy = solana_prices[solana_prices['date'] == date]
    sol_price_1d_ago = solana_prices[solana_prices['date'] == ts_1d_ago]
    sol_price_2d_ago = solana_prices[solana_prices['date'] == ts_2d_ago]
    sol_price_1w_ago = solana_prices[solana_prices['date'] == ts_1w_ago]

    sp500_price_tdy = sp500_prices[sp500_prices['date'] == date]
    sp500_price_1d_ago = sp500_prices[sp500_prices['date'] == ts_1d_ago]
    sp500_price_2d_ago = sp500_prices[sp500_prices['date'] == ts_2d_ago]
    sp500_price_1w_ago = sp500_prices[sp500_prices['date'] == ts_1w_ago]

    eth_price_tdy = ethereum_prices[ethereum_prices['date'] == date]
    eth_price_1d_ago = ethereum_prices[ethereum_prices['date'] == ts_1d_ago]
    eth_price_2d_ago = ethereum_prices[ethereum_prices['date'] == ts_2d_ago]
    eth_price_1w_ago = ethereum_prices[ethereum_prices['date'] == ts_1w_ago]

    btc_price_tdy = bitcoin_prices[bitcoin_prices['date'] == date]
    btc_price_1d_ago = bitcoin_prices[bitcoin_prices['date'] == ts_1d_ago]
    btc_price_2d_ago = bitcoin_prices[bitcoin_prices['date'] == ts_2d_ago]
    btc_price_1w_ago = bitcoin_prices[bitcoin_prices['date'] == ts_1w_ago]

    if sp500_price_tdy.empty or sp500_price_1d_ago.empty or sp500_price_2d_ago.empty or sp500_price_1w_ago.empty:
        continue
    
    if sol_price_tdy.empty or sol_price_1d_ago.empty or sol_price_2d_ago.empty or sol_price_1w_ago.empty:
        continue

    if eth_price_tdy.empty or eth_price_1d_ago.empty or eth_price_2d_ago.empty or eth_price_1w_ago.empty:
        continue
    
    if btc_price_tdy.empty or btc_price_1d_ago.empty or btc_price_2d_ago.empty or btc_price_1w_ago.empty:
        continue
    
    # SP500 has open/close columns, so we can calculate open-to-close changes
    sp500_price_current = sp500_price_tdy.iloc[0]['close']
    sp500_price_1d_change = sp500_price_1d_ago.iloc[0]['close'] - sp500_price_1d_ago.iloc[0]['open']
    sp500_price_2d_change = sp500_price_2d_ago.iloc[0]['close'] - sp500_price_2d_ago.iloc[0]['open']
    sp500_price_1w_change = sp500_price_1w_ago.iloc[0]['close'] - sp500_price_1w_ago.iloc[0]['open']

    # Crypto prices only have close_price, so calculate changes from close_price differences
    # Change = today - past (positive means price went up)
    sol_price_current = sol_price_tdy.iloc[0]['close_price']
    sol_price_1d_change = sol_price_tdy.iloc[0]['close_price'] - sol_price_1d_ago.iloc[0]['close_price']
    sol_price_2d_change = sol_price_tdy.iloc[0]['close_price'] - sol_price_2d_ago.iloc[0]['close_price']
    sol_price_1w_change = sol_price_tdy.iloc[0]['close_price'] - sol_price_1w_ago.iloc[0]['close_price']

    eth_price_current = eth_price_tdy.iloc[0]['close_price']
    eth_price_1d_change = eth_price_tdy.iloc[0]['close_price'] - eth_price_1d_ago.iloc[0]['close_price']
    eth_price_2d_change = eth_price_tdy.iloc[0]['close_price'] - eth_price_2d_ago.iloc[0]['close_price']
    eth_price_1w_change = eth_price_tdy.iloc[0]['close_price'] - eth_price_1w_ago.iloc[0]['close_price']

    btc_price_current = btc_price_tdy.iloc[0]['close_price']
    btc_price_1d_change = btc_price_tdy.iloc[0]['close_price'] - btc_price_1d_ago.iloc[0]['close_price']
    btc_price_2d_change = btc_price_tdy.iloc[0]['close_price'] - btc_price_2d_ago.iloc[0]['close_price']
    btc_price_1w_change = btc_price_tdy.iloc[0]['close_price'] - btc_price_1w_ago.iloc[0]['close_price']

    combined_rows.append({
        'date': date,
        'close_price': sol_close_price,
        'volume': sol_volume,
        '8h_change': sol_8h_change,
        '16h_change': sol_16h_change,
        '24h_change': sol_24h_change,
        'sp500_1d_change': sp500_price_1d_change,
        'sp500_2d_change': sp500_price_2d_change,
        'sp500_1w_change': sp500_price_1w_change,
        'sol_1d_change': sol_price_1d_change,
        'sol_2d_change': sol_price_2d_change,
        'sol_1w_change': sol_price_1w_change,
        'eth_1d_change': eth_price_1d_change,
        'eth_2d_change': eth_price_2d_change,
        'eth_1w_change': eth_price_1w_change,
        'btc_1d_change': btc_price_1d_change,
        'btc_2d_change': btc_price_2d_change,
        'btc_1w_change': btc_price_1w_change,
    })

combined_df = pd.DataFrame(combined_rows)
combined_df.to_csv("data/combined_dataset.csv", index=False)
print(f"Saved combined dataset with {len(combined_df)} rows")


Saved combined dataset with 1899 rows
